In [87]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

#Data
sentences = ["i like dog", "i love coffee", "i hate milk", "you like cat", "you love milk", "you hate coffee"]

i2w = list(set(' '.join(sentences).split()))
w2i = {w:i for i,w in enumerate(i2w)}

In [132]:
#Dataset & Dataloader
class SentenceDataset(Dataset):
    def __init__(self, sentences, w2i):
        self.sentences = sentences
        self.w2i = w2i 

        self.x = []
        self.y = []
        for sen in sentences:
            words = sen.split()
            data_idx = [w2i[w]for w in words[:-1]]
            target = w2i[words[-1]]

            self.x.append(np.eye(len(self.w2i))[data_idx])
            self.y.append(target)
        
        self.x = torch.tensor(self.x).type(torch.FloatTensor) 
        self.y = torch.tensor(self.y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self,idx):
        return self.x[idx], self.y[idx]

#hyperparameters
n_layer = 2
hidden_size = 15
batch_size = 3
lr = 0.01


#dataset
train_data = SentenceDataset(sentences, w2i)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False)

In [133]:
#model 구현
class TextRNN(nn.Module):
    def __init__(self, w2i, sequence_length, hidden_size):
        super().__init__()
        
        self.rnn = nn.RNN(input_size=len(w2i), hidden_size=hidden_size, num_layers=2, batch_first=True)
        self.fc = nn.Linear(sequence_length*hidden_size, len(w2i))
        
    def forward(self,x, hidden):
        out, hn = self.rnn(x,hidden)
        out = torch.flatten(out,1)
        out = self.fc(out)
        return out

In [143]:
#train
model = TextRNN(w2i, n_layer, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

model.train()
h_0 = torch.zeros((n_layer,batch_size,hidden_size))
for epoch in tqdm(range(500),leave=False):
    for input, target in train_loader:
        output = model(input,h_0)
        loss = criterion(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print('Epoch:', '%04d'%(epoch + 1), 'cost =', '{:.6f}'.format(loss))

  0%|          | 0/500 [00:00<?, ?it/s]

Epoch: 0100 cost = 0.015562
Epoch: 0200 cost = 0.004549
Epoch: 0300 cost = 0.002317
Epoch: 0400 cost = 0.001437
Epoch: 0500 cost = 0.000988


In [158]:
#test
test_sentence = ['i love', 'you like', 'i hate']
test = []
for sen in test_sentence:
    words = sen.split()
    input = [w2i[w] for w in words]
    test.append(torch.eye(len(w2i))[input])

test_data = torch.cat(test).view(3,2,-1)

idxs = model(test_data, h_0).max(dim=-1)[1]

for i, idx in enumerate(idxs):
    print(test_sentence[i]+f' -> {i2w[idx]}')

i love -> coffee
you like -> cat
i hate -> milk
